In [1]:
import sys
sys.path.insert(0, 'C:/Users/Lenovo/Desktop/exam_eti/containerized_tool/data_analysis_tool/src')

In [2]:
import mysql_analyzer
import os

In [3]:
mysql_works = mysql_analyzer.mysql_profiler('localhost',os.environ['MYSQLSERVER_USER'],
                os.environ['MYSQLSERVER_PASS'],'sakila')

In [4]:
# mysql_works.multiple_dataset_apply_containing_cols_mysql()

In [5]:
# mysql_works.find_value_mysql('Hillyer')

In [6]:
mysql_works.multiple_dataset_apply_mysql_query('SELECT * FROM criket_table')

[('Pakistan', 'India', 'India'),
 ('SriLanka', 'India', 'India'),
 ('Bangladesh', 'India', 'India'),
 ('SriLanka', 'Pakistan', 'SriLanka'),
 ('Bangladesh', 'Pakistan', 'Bangladesh'),
 ('Bangladesh', 'SriLanka', 'SriLanka'),
 ('India', 'SriLanka', 'India')]

In [7]:
mysql_works.multiple_dataset_apply_mysql_query('SELECT team, COUNT(team) as game_played,\
                                          CAST(SUM(CASE WHEN team=WonBy THEN 1 ELSE 0 END)AS SIGNED) AS game_won FROM\
                                         (SELECT TeamA AS team, WonBy from criket_table\
                                         UNION ALL\
                                         SELECT TeamB AS team, WonBy from criket_table) AS sub\
                                         GROUP BY team\
                                         ORDER BY game_won'
                                         )

[('Pakistan', 3, 0), ('Bangladesh', 3, 1), ('SriLanka', 4, 2), ('India', 4, 4)]